<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет! Меня зовут Тимофей Вилков, и я буду проверять твой проект. Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>

<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>



<div class="alert alert-block alert-success">
✔️ <b>Зеленый текст</b> - все отлично
</div>

<div class="alert alert-block alert-warning">
⚠️ <b>Желтый текст</b> - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание
</div>

<div class="alert alert-block alert-danger">
❌ <b>Красный текст</b> - красный текст - есть недочеты
</div>

---

<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например:

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> 

</div>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-success">
✔️ Молодец, что добавил содержание. Так работать с проектом гораздо удобнее
</div>

In [40]:
user_behavior = pd.read_csv('/datasets/users_behavior.csv')
display(user_behavior.head(5))
print(len(user_behavior))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


3214


<div class="alert alert-block alert-success">
✔️ Данные на месте, отлично!
</div>

<div class="alert alert-block alert-warning">
⚠️ Советую проводить хотя бы минимальный EDA перед обучением модели, понимание структуры/особенностей/качества данных способствует улучшению качества модели
</div>

## Разбейте данные на выборки

In [7]:
features = user_behavior.drop('is_ultra',axis=1)
target = user_behavior['is_ultra']
train_X, test_X, train_y, test_y = train_test_split(features,target,random_state=12345,test_size=0.25)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,random_state=12345,test_size=0.25)

<div class="alert alert-block alert-success">
✔️ С соотношением согласен, но также советую использовать стратификацию по целевой переменной (параметр stratify)
</div>

<div class="alert alert-block alert-success">
✔️ Разбиение выборки на обучающую, валидационную и тестовую реализовано правильно
</div>

## Исследуйте модели

In [35]:
# Random Forest Classifier
best_model_rf = None
best_result = 0
for est in range(1, 11):
    for depth in range(1, 6):
        for feature in ['auto', 'sqrt']:
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth,max_features=feature)
            model.fit(train_X,train_y)
            result = model.score(valid_X,valid_y)
    if result > best_result:
        best_model_rf = model
        best_result = result
print(f'Accuracy наилучшей модели на валидационной выборке:{best_result}')
print(f'Лучшая модель:{best_model_rf}')

Accuracy наилучшей модели на валидационной выборке:0.8092868988391376
Лучшая модель:RandomForestClassifier(max_depth=5, max_features='sqrt', n_estimators=9,
                       random_state=12345)


<div class="alert alert-block alert-success">
✔️ Обучение и подбор гиперпараметров реализованы верно, молодец!
</div>

In [36]:
# Decision Tree Classifier
best_model_dt = None
best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(train_X,train_y)
    result = model.score(valid_X,valid_y)
    if result > best_result:
        best_model_dt = model
        best_result = result
print(f'Accuracy наилучшей модели на валидационной выборке:{best_result}')
print(f'Лучшая модель:{best_model_dt}')

Accuracy наилучшей модели на валидационной выборке:0.7943615257048093
Лучшая модель:DecisionTreeClassifier(max_depth=3, random_state=12345)


<div class="alert alert-block alert-success">
✔️ Также можно получать оценки принадлежности классам с помощью <code>predict_proba()</code> и настраивать порог для бинаризации самостоятельно. Это может дать существенный прирост в метрике. По умолчанию (в predict) порогом является 0.5 
</div>

In [37]:
# Logistic Regression
best_model_lr = None
best_result = 0
for solv in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    for iterations in range(250,3000,250):
        model = LogisticRegression(random_state=12345, solver=solv,max_iter=iterations)
        model.fit(train_X,train_y)
        result = model.score(valid_X,valid_y)
    if result > best_result:
        best_model_lr = model
        best_result = result
print(f'Accuracy наилучшей модели на валидационной выборке:{best_result}')
print(f'Лучшая модель:{best_model_lr}')

Accuracy наилучшей модели на валидационной выборке:0.7412935323383084
Лучшая модель:LogisticRegression(max_iter=2750, random_state=12345, solver='newton-cg')


<div class="alert alert-block alert-warning">
⚠️  По умолчанию LogisticRegression реализует алгоритм с регуляризацией (penalty='l2'), чтобы решение было корректным нужно привести признаки к одному масштабу. Если же хочешь использовать классический вариант логистической регрессии стоит использовать penalty='none'
</div>

## Проверьте модель на тестовой выборке

In [38]:
print(f'Accuracy Random Forest Classifier на тестовой выборке:{best_model_rf.score(test_X,test_y)}')
print(f'Accuracy Decision Tree Classifier на тестовой выборке:{best_model_dt.score(test_X,test_y)}')
print(f'Accuracy Logistic Regression на тестовой выборке:{best_model_lr.score(test_X,test_y)}')

Accuracy Random Forest Classifier на тестовой выборке:0.7922885572139303
Accuracy Decision Tree Classifier на тестовой выборке:0.7910447761194029
Accuracy Logistic Regression на тестовой выборке:0.7611940298507462


Как и на валиационной выборке лучше всего отработала модель Random Forest Classifier с accuracy 79,2%. Однако, Decision Tree Classifier отстал совсем немного на ~0.13%. Это говорит о том, что глубоких лесов нам для наших данных не нужно, достаточно одного дерева) Также, минимальная разница в accuracy на валидационной и тестовой выборках говорит о том, что данные не переобучены и не недообучены.

<div class="alert alert-block alert-success">
✔️ Необходимое значение метрики достигнуто!
</div>

## (бонус) Проверьте модели на адекватность

Не понял, что от меня требуется сделать...

<div class="alert alert-block alert-success">
✔️ На мой взгляд, проверка на адекватность — это попытка понять как работает модель (её интерпретация), а не подсчёт классических метрик. Для линейных моделей можно интерпретировать работу модели через величину коэффициентов (только признаки нужно отшкалировать), для деревянных ансамблей через feature importance. Также есть различные техники позволяющие интепретировать любой чёрный ящик, например, shap values или permutation feature importance 
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75


<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'>Нурлан, отличный проект, видно, что ты можешь много показать. Чистый, аккуратный код и отличная структура. Модели машинного обучения верно применены (кроме нюанса с регуляризацией), необходимое качество целевой метрики достигнуто. Прикрепил статью, где подробно разобран дисбаланс классов для задачи бинарной классификации, в нём нет ничего критичного. В целом работа хорошая, серьёзных замечаний у меня нет:)
Я верну тебе проект чтобы ты мог задать вопросы, если их нет, сразу по возвращении зачту его. 

<div class="alert alert-block alert-info">
<b>Спасибо за feedback, вопросов нет.</b> 

</div>

<div class="alert alert-block alert-success">
✔️ Перевожу твой проект в статус "зачтено" и желаю дальнейших успехов в освоении курса!
</div>